In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df_raw = pd.read_excel("data/gunn-kinzer-raindrop-velocity-small.xlsx")
df_raw.head()

AttributeError: module 'pandas' has no attribute 'read_xlsx'